In [41]:
import re
from datetime import datetime
from calendar_ipynb.ipywidgets.calendar_selection import (
    select_calendars,
    get_selected_calendars,
)
from calendar_ipynb.events import fetch_events_parallel

EMAIL_IDS = ["fahimalizain@gmail.com", "fahim.zain@kwalee.com"]
select_calendars(EMAIL_IDS)

INFO:googleapiclient.discovery_cache:file_cache is only supported with oauth2client<4.0.0
INFO:googleapiclient.discovery_cache:file_cache is only supported with oauth2client<4.0.0


In [44]:
from calendar_ipynb.events import get_calendar_service


TITLE_REGEX = r"^(.*Venniyur.*) \| Family$"
RENAME_REGEX = r"\1 | KK-Relatives"

from_datetime = datetime.fromisoformat("2025-01-01T00:00:00Z".replace("Z", "+00:00"))
to_datetime = datetime.fromisoformat("2026-01-01T23:59:59Z".replace("Z", "+00:00"))
print(get_selected_calendars())

all_events = fetch_events_parallel(
    email_map=get_selected_calendars(),
    from_datetime=from_datetime,
    to_datetime=to_datetime
)

matcher = re.compile(TITLE_REGEX)
EVENTS_TO_RENAME = []
for event in all_events:
    if not matcher.match(event['summary']):
        continue

    if event.get("status", None) != "confirmed":
        continue

    if event.get("eventType", None) not in ["default", "fromGmail"]:
        continue

    if "creator" in event and event["creator"].get("email", None) != event["email"]:
        continue

    EVENTS_TO_RENAME.append(event)

print(f"Found {len(EVENTS_TO_RENAME)} events to rename")
count_by_summary = {}
for event in EVENTS_TO_RENAME:
    count_by_summary[event['summary']] = count_by_summary.get(event['summary'], 0) + 1

unique_summaries = list(set(event['summary'] for event in EVENTS_TO_RENAME))
unique_summaries.sort(key=lambda s: count_by_summary[s], reverse=True)
for summary in unique_summaries:
    print(f" - x{count_by_summary[summary]} '{summary}' --> '{re.sub(TITLE_REGEX, RENAME_REGEX, summary)}'")

{'fahimalizain@gmail.com': ['esvag4fbb8q192q3ksipqh5d2g@group.calendar.google.com', 'j9d5885raedlop9apnj5c4odoo@group.calendar.google.com', 'mbn0dk4phhhufr6hpbrl15p4h0@group.calendar.google.com', 'fahimalizain@gmail.com'], 'fahim.zain@kwalee.com': ['fahim.zain@kwalee.com']}


INFO:googleapiclient.discovery_cache:file_cache is only supported with oauth2client<4.0.0
INFO:googleapiclient.discovery_cache:file_cache is only supported with oauth2client<4.0.0
INFO:googleapiclient.discovery_cache:file_cache is only supported with oauth2client<4.0.0
INFO:googleapiclient.discovery_cache:file_cache is only supported with oauth2client<4.0.0
INFO:googleapiclient.discovery_cache:file_cache is only supported with oauth2client<4.0.0
INFO:calendar_ipynb.events_incremental:Sync completed for fahimalizain@gmail.com/mbn0dk4phhhufr6hpbrl15p4h0@group.calendar.google.com: Added 0, Updated 0, Deleted 0
INFO:calendar_ipynb.events_incremental:Sync completed for fahimalizain@gmail.com/j9d5885raedlop9apnj5c4odoo@group.calendar.google.com: Added 0, Updated 0, Deleted 0
INFO:calendar_ipynb.events_incremental:Sync completed for fahimalizain@gmail.com/fahimalizain@gmail.com: Added 0, Updated 0, Deleted 0
INFO:calendar_ipynb.events_incremental:Sync completed for fahim.zain@kwalee.com/fahim

Found 5 events to rename
 - x1 'Parappanangadi Beach / Venniyur | Family' --> 'Parappanangadi Beach / Venniyur | KK-Relatives'
 - x1 'Venniyur / Pick Up Aawu & Aza | Family' --> 'Venniyur / Pick Up Aawu & Aza | KK-Relatives'
 - x1 'Pick Aawu from Venniyur | Family' --> 'Pick Aawu from Venniyur | KK-Relatives'
 - x1 'Kondotty to Venniyur | Family' --> 'Kondotty to Venniyur | KK-Relatives'
 - x1 'Venniyur | Family' --> 'Venniyur | KK-Relatives'


In [43]:
raise Exception("Stopping here to avoid accidental renaming. Manually invoke the renaming code below if you are sure.")

Exception: Stopping here to avoid accidental renaming. Manually invoke the renaming code below if you are sure.

In [45]:
SERVICE_MAP = {
    email: get_calendar_service(email) for email in EMAIL_IDS
}

for event in EVENTS_TO_RENAME:
    original_summary = event['summary']
    new_summary = re.sub(TITLE_REGEX, RENAME_REGEX, original_summary)

    service = SERVICE_MAP[event['email']]
    service.events().update(
        calendarId=event["calendar_id"], eventId=event["id"], body=dict(event, summary=new_summary)
    ).execute()
    print("Renamed", f"'{original_summary}'", "to", f"'{new_summary}'", event["start"])

INFO:googleapiclient.discovery_cache:file_cache is only supported with oauth2client<4.0.0
INFO:googleapiclient.discovery_cache:file_cache is only supported with oauth2client<4.0.0


Renamed 'Parappanangadi Beach / Venniyur | Family' to 'Parappanangadi Beach / Venniyur | KK-Relatives' {'dateTime': '2025-04-20T05:45:00+05:30', 'timeZone': 'Asia/Kolkata'}
Renamed 'Pick Aawu from Venniyur | Family' to 'Pick Aawu from Venniyur | KK-Relatives' {'dateTime': '2025-04-20T06:45:00+05:30', 'timeZone': 'Asia/Kolkata'}
Renamed 'Venniyur | Family' to 'Venniyur | KK-Relatives' {'dateTime': '2025-04-20T10:15:00+05:30', 'timeZone': 'Asia/Kolkata'}
Renamed 'Venniyur / Pick Up Aawu & Aza | Family' to 'Venniyur / Pick Up Aawu & Aza | KK-Relatives' {'dateTime': '2025-05-18T06:15:00+05:30', 'timeZone': 'Asia/Kolkata'}
Renamed 'Kondotty to Venniyur | Family' to 'Kondotty to Venniyur | KK-Relatives' {'dateTime': '2025-06-24T17:55:00+05:30', 'timeZone': 'Asia/Kolkata'}
